# QA Image QwenVL 2b

In [1]:
import os
import sys

sys.path.append(os.path.abspath(".."))

from transformers import Qwen3VLForConditionalGeneration, AutoProcessor
from clear_vision.config.decorators import singleton
from utils import extract_frames


@singleton
class QwenModel:
    def __init__(self) -> None:
        self.model = Qwen3VLForConditionalGeneration.from_pretrained(
            "Qwen/Qwen3-VL-2B-Instruct", dtype="auto", device_map="auto"
        )
        self.processor = AutoProcessor.from_pretrained("Qwen/Qwen3-VL-2B-Instruct")


qwen = QwenModel()

/home/rapha/.cache/pypoetry/virtualenvs/clear-vision-backend-pHRXtyny-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from PIL import Image

video_path = os.path.abspath('../public/video_rua01.mp4')
frames, ts = extract_frames(video_path)

frame = Image.fromarray(frames[0])

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": frame},
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# Preparation for inference
inputs = qwen.processor.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_dict=True,
    return_tensors="pt"
)
inputs = inputs.to(qwen.model.device)

# Inference: Generation of the output
generated_ids = qwen.model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = qwen.processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

# TOO SLOW!!!!
print(output_text)

['This is a black and white photograph of a city street, likely in New York City, captured from a low angle looking down the street. The scene is characterized by a strong sense of motion and urban life.\n\n- **Composition and Perspective**: The photograph is taken from a street-level viewpoint, looking down a wide avenue. The perspective is slightly tilted, creating a dynamic feel. The street is lined with multi-story buildings on both sides, featuring classic architectural details such as arched windows, fire escapes, and ornate facades. The buildings on the left are more ornate and older, while those on the right are more modern.\n\n-']
